**LABORATORIO 3 SIS420**

Arancibia Aguilar Daniel Andree

**CLASIFICACIÓN MULTICLASE**

La clasificación multiclase es un tipo de problema en el aprendizaje automático donde tenemos que asignar una instancia como una imagen o texto a una de varias clases diferentes. En lugar de tener solo dos opciones como en la clasificación binaria, en la clasificación multiclase tenemos más opciones entre las cuales elegir.

Usare un dataset de reconocimiento de letras con 26 clases en las que cada letra equivale a una clase

https://www.kaggle.com/datasets/nishan192/letterrecognition-using-svm

El dataset cuenta con 17 columnas y 20000 filas, se cumple el requisito para el laboratorio n>10 y m>=20000

Valores de entrada:
xbox

ybox

width

height

onpix

xbar

ybar

x2bar

xybar

x2ybar

xy2bar

xedge

xedgey

yedgex

Valor de salida:
letter


**Importamos el drive**

In [190]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Importamos todas las librerias que usaremos en nuestro cuadernillo**

In [189]:
import os
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn.svm import SVC
from matplotlib import pyplot
from scipy import optimize
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
%matplotlib inline

**Empiezo cargando el dataset en PANDAS esto para hacerle un análisis y modificación que hara que funcione correctamente para la Clasificación Multiclase**

In [191]:
url = "/content/gdrive/MyDrive/SIS420/Lab3/letter-recognition.csv"
dataframe = pd.read_csv(url)
print(dataframe)

      letter  xbox   ybox   width   height  onpix   xbar   ybar   x2bar  \
0          T      2      8       3       5       1      8     13      0   
1          I      5     12       3       7       2     10      5      5   
2          D      4     11       6       8       6     10      6      2   
3          N      7     11       6       6       3      5      9      4   
4          G      2      1       3       1       1      8      6      6   
...      ...    ...    ...     ...     ...     ...    ...    ...    ...   
19995      D      2      2       3       3       2      7      7      7   
19996      C      7     10       8       8       4      4      8      6   
19997      T      6      9       6       7       5      6     11      3   
19998      S      2      3       4       2       1      8      7      2   
19999      A      4      9       6       6       2      9      5      3   

       y2bar   xybar   x2ybar  xy2bar  xedge   xedgey  yedge   yedgex  
0           6       6      

**Análisis del dataframe para verificar si existen valores nulos y poder ver el tamaño del dataframe**

En el caso de mi dataframe no se encontró valores nulos que afecten al laboratorio

In [192]:
print(dataframe.describe())

              xbox          ybox         width        height        onpix   \
count  20000.000000  20000.000000  20000.000000  20000.00000  20000.000000   
mean       4.023550      7.035500      5.121850      5.37245      3.505850   
std        1.913212      3.304555      2.014573      2.26139      2.190458   
min        0.000000      0.000000      0.000000      0.00000      0.000000   
25%        3.000000      5.000000      4.000000      4.00000      2.000000   
50%        4.000000      7.000000      5.000000      6.00000      3.000000   
75%        5.000000      9.000000      6.000000      7.00000      5.000000   
max       15.000000     15.000000     15.000000     15.00000     15.000000   

              xbar          ybar          x2bar        y2bar         xybar   \
count  20000.000000  20000.000000  20000.000000  20000.000000  20000.000000   
mean       6.897600      7.500450      4.628600      5.178650      8.282050   
std        2.026035      2.325354      2.699968      2.38082

**Modificación de valores**

En mi dataset la primera columna "letter" tenia valores de texto desde la A - Z que no son compatibles con el modelo

Modifico todas las letras cambiandolas por un valor numérico del 1 al 26 que representa el numero de clase

In [193]:
mapeo_letras = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10,
                'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19,
                'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}
dataframe['letter'] = dataframe['letter'].map(mapeo_letras)

# Imprimo el DataFrame resultante con la columna 'letter' modificada
print(dataframe)

       letter  xbox   ybox   width   height  onpix   xbar   ybar   x2bar  \
0          20      2      8       3       5       1      8     13      0   
1           9      5     12       3       7       2     10      5      5   
2           4      4     11       6       8       6     10      6      2   
3          14      7     11       6       6       3      5      9      4   
4           7      2      1       3       1       1      8      6      6   
...       ...    ...    ...     ...     ...     ...    ...    ...    ...   
19995       4      2      2       3       3       2      7      7      7   
19996       3      7     10       8       8       4      4      8      6   
19997      20      6      9       6       7       5      6     11      3   
19998      19      2      3       4       2       1      8      7      2   
19999       1      4      9       6       6       2      9      5      3   

       y2bar   xybar   x2ybar  xy2bar  xedge   xedgey  yedge   yedgex  
0           6  

**Convertir el dataset modificado en PANDAS**

Usare la variable data para convertir el dataset a un array compatible con numpy que guarde solo los valores

In [157]:
data = dataframe.values
X, y = data[:, 1:], data[:, 0]
#Imprimo el tamaño de las caracteristicas y del label que usare
print(X.shape)
print(y.shape)

(20000, 16)
(20000,)


In [158]:
#Imprimo y para ver el array
y

array([20,  9,  4, ..., 20, 19,  1])

**Defino variables que coincidan con mi dataset para usarlas en las funciones**

In [159]:
input_layer_size  = 17
num_labels = 26
m = y.size

**Defino la funcion para calcular la sigmoide **

Esta funcion nos servira para hacer las predicciones

In [160]:
def calcularSigmoide(z):
    return 1.0 / (1.0 + np.exp(-z))

**Defino la funcion calcularCosto**

La usar para evaluar y mejorar la precisión del modelo al manejar múltiples clases

In [161]:
def calcularCosto(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = calcularSigmoide(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    # Se aplica regularizacion en la siguiente linea
    grad = grad + (lambda_ / m) * temp

    return J, grad

Defino valores de prueba para la funcion de calcularCosto

In [194]:
#Valor de prueba para theta
theta_t = np.array([0.5, 0.2, 0.8, 0.3, 0.1,0.4, 0.7, 0.1, 0.2, 0.5, 0.6, 0.6, 0.3, 0.7, 0.4, 0.8,0.9], dtype=float)

# Valores de prueba para las características
# Concateno una columna de unos
X_t = np.concatenate((np.ones((15, 17)), np.arange(1, 16).reshape(15, 1, order='F')/10.0), axis=1)

# Valores de prueba para las entradas X
X_t = np.concatenate((np.ones((20000, 1)), np.random.rand(20000, 16)), axis=1)
print(X_t)

# Valores de prueba para las clases
y_t = np.random.randint(0, 26, 20000)

# Valor de prueba para la regularización
lambda_t = 0.2

[[1.         0.74291169 0.78348988 ... 0.13398965 0.9866334  0.59014541]
 [1.         0.37137195 0.30951346 ... 0.63205062 0.73145424 0.00909578]
 [1.         0.3137744  0.98988815 ... 0.06421577 0.34199219 0.17276181]
 ...
 [1.         0.34131595 0.48785605 ... 0.16259161 0.05801449 0.41830987]
 [1.         0.66898618 0.89812529 ... 0.14212825 0.13568148 0.62632146]
 [1.         0.41592231 0.77475351 ... 0.93437375 0.58467432 0.51645016]]


**Prueba de la funcion calcularCosto**

In [195]:
J, grad = calcularCosto(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : -49.266894
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [-11.505162, -5.783847, -5.692062, -5.704354]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


**Defino la funcion oneVsAll**

La función OnevsAll permite entrenar múltiples clasificadores binarios para resolver problemas de clasificación multiclase y combinar sus predicciones para obtener una clasificación final

Esta funcion hace que se cree una clasificación binaria para cada clase que tengamos en el dataset en el que se trata la clase como una positiva y a las demas como negativas

In [164]:
def oneVsAll(X, y, num_labels, lambda_):

    m, n = X.shape
    all_theta = np.zeros((num_labels, n + 1))
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 5000}
        res = optimize.minimize(calcularCosto,
                                initial_theta,
                                (X, (y == (c + 1)), lambda_),
                                jac=True,
                                method='BFGS',
                                options=options)
        all_theta[c] = res.x
    return all_theta


**Establezco la variable de regularización (lambda) y usamos la funcion onevsall para obtener las thetas**

In [165]:
lambda_ = 0.2
all_theta = oneVsAll(X, y, num_labels, lambda_)

In [196]:
#Imprimo las thetas
print(all_theta)

[[ 6.33431855e+00 -7.93078727e-01  1.71746765e-01  9.48671497e-01
   2.73110090e-02 -5.20882996e-01  5.72807754e-01 -3.50921822e-01
  -4.97150153e-01 -1.02436910e+00 -6.27195244e-01 -3.86999257e-01
   6.17320260e-01 -1.75930324e-01 -1.80721798e-01  2.32460166e-01
  -6.96378818e-01]
 [-2.27905635e+00  5.36612851e-02  5.65606807e-02 -6.00894455e-01
  -7.47765277e-01  8.76621210e-01  6.54983139e-02 -3.82160826e-01
   2.82399988e-01  2.09455492e-01  3.21706230e-01  1.38264434e-01
  -9.18969381e-01 -1.12420624e-01 -3.58197684e-01  5.63471650e-01
   5.74714769e-01]
 [-3.56443609e+01  1.86346775e-01  1.78634655e-01 -3.38025155e-01
  -4.36337319e-01  1.43722402e-01  9.98926611e-02  4.72703137e-01
   3.42332191e-01  4.31513312e-01  4.38784847e-01  2.29627189e-01
   1.04794835e+00  3.29056725e-01  4.40080339e-01 -3.65121062e-02
   6.02534454e-01]
 [ 2.95191054e+00 -5.26692953e-02  2.13608540e-01 -9.15572005e-01
  -9.73321395e-01  1.25957676e+00  2.21597589e-03 -6.26345806e-01
   8.14006940e-01  

**Defino la funcion predictOneVsAll**

Esta funcion se encargará de hacer las predicciones haciendo uso de las thetas y las caracteristicas X

In [167]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(calcularSigmoide(X.dot(all_theta.T)), axis = 1)

    return p + 1

**Separo las características del dataset para el entrenamiento (80%) y el test(20%)**

Esto usando train_test_split

In [187]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
#Imprimo el tamaño de los datos de entrenamiento y test
print(X_train.shape)
print(X_test.shape)

(16000, 16)
(4000, 16)


**Probamos la precisión del dataset completo usando classifier visto en PANDAS**

In [182]:

classifier =  SVC()

classifier.fit(X_train, y_train)

score = classifier.score(X_test, y_test)*100
print('Precisión del conjunto: {:.2f}%'.format(score))

Precisión del conjunto: 92.20%


**Hacemos las pruebas de precisión y predicciones**

Se calcula la precision del entrenamiento (X_test) usando mean de numpy en que la que obtuvimos una precisión de 73.12%

Hacemos las predicciones con los datos de X_test y usando la funcion argmax

In [208]:
pred_train = predictOneVsAll(all_theta, X_train)
#calculamos la precisión del entrenamiento (80%)
precision_entrenamiento = np.mean(pred_train == y_train) * 100
print('Precisión del conjunto de entrenamiento: {:.2f}%'.format(precision_entrenamiento))

XPrueba = X_test[:100].copy()
yPrueba = y_test.copy()
#concatenamos la columna de unos a XPrueba
XPrueba = np.concatenate((np.ones((100, 1)), XPrueba), axis=1)
print(XPrueba)
#Hacemos las predicciones del test (20%)
pred_prueba = np.argmax(calcularSigmoide(XPrueba.dot(all_theta.T)), axis=1)

print('Predicciones en el conjunto de prueba:')
print(pred_prueba + 1)


Precisión del conjunto de entrenamiento: 73.12%
[[ 1.  3.  8. ...  8.  7.  7.]
 [ 1.  2.  7. ...  6.  0.  8.]
 [ 1.  2.  4. ... 10.  2.  5.]
 ...
 [ 1.  1.  3. ...  8.  1.  8.]
 [ 1.  2.  0. ...  8.  0.  8.]
 [ 1.  3.  6. ...  7.  7.  8.]]
Predicciones en el conjunto de prueba:
[ 5 10 20 22 12 15 14  6  7 10  4 11  2 19  3 23 21  9  7 14  5 22 24  7
 25 21 12  8 16 13 13  6 11  8 17 14 19  1 20  3 16  3  2 25  7  9 17  2
  3  3 20  7 20  3 26 21 11 18 13 20  3 17  8 24  4 13 21  8  3 11  9 18
 15 11 26  1  3 21  9 20  4  3  1 10 12 23 22  3 25 25 17 13 26 23 21 23
 11  9 12 18]


**Hice una prueba con datos de una fila del dataset y se pudo ver que coincide con la letra del dataset**

In [207]:
# Datos para prueba (fila 28)
datos = [ [12,	14,	12,	8,	5,	9,	10,	4,	3,	5,	10,	7,	10,	12	,2,	6]]
XX = np.array(datos)

XPrueba = XX.copy()
yPrueba = y_test.copy()
#concatenamos la columna de unos a XPrueba
XPrueba = np.concatenate((np.ones((1, 1)), XPrueba), axis=1)
print(XPrueba)
#Hacemos las predicciones del test (20%)
pred_prueba = np.argmax(calcularSigmoide(XPrueba.dot(all_theta.T)), axis=1)

print('Prediccion en la prueba:', pred_prueba + 1)
print('Prediccion esperada: 23')

[[ 1. 12. 14. 12.  8.  5.  9. 10.  4.  3.  5. 10.  7. 10. 12.  2.  6.]]
Prediccion en la prueba: [23]
Prediccion esperada: 23


**CONCLUSIONES**

Con el dataset que use pude hacer las modificaciones necesarias en PANDAS para ser usado en la Clasificación Multiclase que separa cada clase de las demás para convertilas en una clasificación binaria

Haciendo uso de las funciones vistas en clase pude obtener una precisión de 92.20% con los datos completos del dataset y un 73.12% de precision con el 80% de datos usados para el entrenamiento

Hice 100 predicciones de test con el 20% de los datos y una extra donde pude comprobar que se predecia la letra correcta usando los mismos valores